In [ ]:
import csv
import numpy as np
import utils_deck_generation as idg
import healpy_pointings as hpoint
import netcdf_read_write as nrw

In [ ]:
imap_nside = 256
imap_npix = imap_nside**2 * 12

# pointings
filename_pointing = "pointing_index_"
surface_cover_radians = np.radians(30.0)
num_pointing = 20

# defocus
filename_defocus = "defocus_index_"
defocus_range = 10.0 # mm
num_defocus = 10

#power
power_range = 0.5 # fraction of full power
num_powers = 10

run_sims = True
run_compression = True
run_type = "nif" # "test"
num_quads = 24

figure_location = "plots"

## Import NIF: Beam, Quad and Cone data

In [ ]:
the_data = dict()

the_data['nbeams'] = 4
the_data['target_radius'] = 1100.0
the_data['facility'] = "NIF"
the_data['nbeams_total'] = 192
the_data['num_quads'] = num_quads

In [ ]:
filename = "NIF_UpperBeams.txt"
j = -1

f=open(filename, "r")
reader = csv.reader(f, delimiter='\t')
for row in reader:
    if j==-1:
        key = row
        for i in range(len(row)):
            the_data[row[i]] = [None] * int(the_data['nbeams_total'])
    else:
        for i in range(len(row)):
            if i < 2:
                the_data[key[i]][j] = row[i]
            elif i < 5:
                the_data[key[i]][j] = float(row[i])
            else:
                the_data[key[i]][j] = int(row[i])
    j=j+1
f.close()
filename = "NIF_LowerBeams.txt"
f=open(filename, "r")
reader = csv.reader(f, delimiter='\t')
for row in reader:
    if j>96:
        for i in range(len(row)):
            if i < 2:
                the_data[key[i]][j-1] = row[i]
            elif i < 5:
                the_data[key[i]][j-1] = float(row[i])
            else:
                the_data[key[i]][j-1] = int(row[i])
    j=j+1
f.close()

the_data["Theta"] = np.radians(the_data["Theta"])
the_data["Phi"] = np.radians(the_data["Phi"])

In [ ]:
the_data['quad_from_each_cone'] = ('Q15T', 'Q13T', 'Q14T', 'Q11T')

## Run Quad and Rotate to form Cone

In [ ]:
the_data["p0"] = [1./4.] * the_data["nbeams"]
the_data["fuse"] = [False] * the_data["nbeams"]

defocus_array = np.linspace(0.0, defocus_range, num_defocus)

for quad_name in the_data['quad_from_each_cone']:
    quad_slice = slice(the_data["Quad"].index(quad_name),the_data["Quad"].index(quad_name)+4)
    beam_names = the_data['Beam'][quad_slice]

    ## Create Healpy grid of pointings
    x, y, z, theta_offset = hpoint.theta_pointing_rotations(the_data, quad_slice, num_pointing, surface_cover_radians)
    r = np.sqrt(x**2 + y**2 + z**2)
    pointing_theta = np.arccos(z/r)
    pointing_phi = np.arctan2(y,x)
    
    for dind in range(len(defocus_array)):
        the_data["defocus"] = [defocus_array[dind]] * the_data["nbeams"]
        for pind in range(len(theta_offset)):
            ## pointing data
            pointings=dict()
            for __name in beam_names:
                __x = float(x[pind])
                __y = float(y[pind])
                __z = float(z[pind])
                pointings[__name]=np.array((__x,__y,__z))
            the_data['pointings'] = []
            for beam in beam_names:
                for key, value in pointings.items():
                    if (key == beam):
                        the_data['pointings'].append(np.asarray(value))

            run_location = filename_pointing + str(pind) + "_" + filename_defocus + str(dind)
            idg.generate_input_deck(the_data, run_location)
            idg.generate_input_pointing_and_pulses(the_data, run_location, quad_slice, run_type)

            if run_sims:
                b='Running ifriit in: '+run_location+"  "
                print("\r", b, end="")
                !cd $run_location;chmod +x main;mpirun -np 1 ./main > a.out

            ## Compression of cone
            if run_compression:
                hpxmap, n_beams = nrw.read_intensity(run_location, run_type, beam_names, imap_nside)
                nrw.rotate_cone_and_save(the_data, quad_name, hpxmap, imap_nside, pointing_theta[pind], pointing_phi[pind], run_location)

## Diagnostic

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import netcdf_read_write as nrw

In [ ]:
pind = 2
dind = 0
run_location = filename_pointing + str(pind) + "_" + filename_defocus + str(dind)

quad_name = 'Q15T'
quad_slice = slice(the_data["Quad"].index(quad_name),the_data["Quad"].index(quad_name)+4)
beam_names = the_data['Beam'][quad_slice]

hpxmap, n_beams = nrw.read_intensity(run_location, run_type, beam_names, imap_nside)

r = the_data['target_radius'] / 1.0e4 # microns to cm
total_TW = np.mean(hpxmap)*10**(-12)
surface_area = 4.0*np.pi*r**2

#rms
avg_power = np.mean(hpxmap)
hpxmap_rms = 100.0 * np.sqrt(np.mean((hpxmap / avg_power - 1.0)**2))
hpxmap_rms_spatial = 100.0 * np.sqrt((hpxmap / avg_power - 1.0)**2)

print('')
print('The total power deposited is ', total_TW*surface_area, 'TW')
print('The power per beam deposited is ', total_TW*surface_area/n_beams, 'TW')
print('This is a drive efficiency of ', total_TW*surface_area/(n_beams*0.25)*100, '%')
print('Number of beams is ', n_beams)
#print('RMS is ', hpxmap_rms, '%')
print('')

hp.mollview(hpxmap,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()

port_theta = np.mean(the_data['Theta'][quad_slice])
port_phi = np.mean(the_data['Phi'][quad_slice])
hp.projscatter(port_theta, port_phi)
quad_pointing_theta = port_theta + theta_offset[pind]
quad_pointing_phi = port_phi
hp.projscatter(quad_pointing_theta, quad_pointing_phi)

## Check cone compression

In [ ]:
from netCDF4 import Dataset
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

In [ ]:
pointing_per_cone = [0,0,0,0]
defocus_per_cone = [0,0,0,0]
power_per_cone = [0,0,0,0]

Y_train = np.hstack((np.array(pointing_per_cone)/num_pointing, np.array(defocus_per_cone)/num_defocus))
Y_train = np.hstack((Y_train, np.array(power_per_cone)/num_powers))
Y_norms = [num_pointing, num_defocus, num_powers]
intensity_map, port_loc, quad_pointing = nrw.assemble_full_sphere(Y_train, Y_norms, the_data, filename_pointing, filename_defocus, power_range)

hp.mollview(intensity_map, unit=r"$\rm{W/cm^2}$", flip="geo")
hp.graticule()

hp.projscatter(port_loc[0,:], port_loc[1,:])
hp.projscatter(quad_pointing[0,:], quad_pointing[1,:])

#quad_name = "Q45B"
#quad_slice = slice(the_data["Quad"].index(quad_name),the_data["Quad"].index(quad_name)+4)
#bottom_portloc_theta = np.mean(the_data['Theta'][quad_slice])
#bottom_portloc_phi = np.mean(the_data['Phi'][quad_slice])
#hp.projscatter(bottom_portloc_theta, bottom_portloc_phi)
plt.savefig(figure_location+'/intensity_mollweide.png', dpi=300, bbox_inches='tight')

## Incident fraction, RMS and modes

In [ ]:
import utils_intensity_map as uim

In [ ]:
intensity_map_rms_spatial = uim.readout_intensity(the_data, intensity_map)

hp.mollview(intensity_map_rms_spatial,unit="RMS",flip="geo")
plt.savefig(figure_location+'/rms_mollweide.png', dpi=300, bbox_inches='tight')
hp.graticule()

In [ ]:
LMAX = 30
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = 20

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes LLE")
plt.xlabel("l mode")
plt.ylabel(r"rms amplitude ($\%$)")
plt.savefig(figure_location+"/unweighted_modes.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"rms amplitude ($\%$)")
plt.savefig(figure_location+"/weighted_modes.png", dpi=300, bbox_inches='tight')

In [ ]:
Y_train1, Y_norms = uim.create_ytrain(pointing_per_cone, num_pointing, defocus_per_cone, num_defocus, power_per_cone, num_powers)

intensity_map, _, _ = nrw.assemble_full_sphere(Y_train, Y_norms, the_data, filename_pointing, filename_defocus, power_range)

intensity_map_normalized, avg_power = uim.imap_norm(intensity_map)
hp.mollview(intensity_map_normalized, unit="Deviation from Mean", flip="geo")

X_train = uim.imap2xtrain(intensity_map_normalized, LMAX)
intensity_map_normalized2 = uim.xtrain2imap(X_train, LMAX, imap_nside)
hp.mollview(intensity_map_normalized2, unit="Deviation from Mean",flip="geo")

# Create data structure for data set and label

## Setup Dataset Constants

In [ ]:
from netCDF4 import Dataset
from os import path
import os
import healpy as hp

In [ ]:
savename_trainingdata = "training_data_and_labels.nc"
LMAX = 30 
num_cones = 4
num_variables = 3
num_coeff = int(((LMAX + 2) * (LMAX + 1))/2.0)
num_examples = num_pointing * num_defocus * num_powers * num_cones
num_output = num_cones * num_variables
Y_norms = [num_pointing, num_defocus, num_powers]
print(num_coeff*2, num_examples, num_output)

## Create and Save Dataset

In [ ]:
nrw.save_training_data(the_data, num_cones, num_pointing, num_defocus, num_powers, num_coeff, num_output, num_examples, power_range, LMAX, savename_trainingdata, filename_pointing, filename_defocus)

## Load Data Set

In [ ]:
training_data = Dataset(savename_trainingdata)
X_train_ordered = training_data.variables["X_train"][:]
Y_train_ordered = training_data.variables["Y_train"][:]
training_data.close()

print(np.shape(X_train_ordered), np.shape(Y_train_ordered))

In [ ]:
test_size = int(num_examples / 10)
index_shuf = list(range(num_examples))

rng = np.random.default_rng(12345)
rng.shuffle(index_shuf)
index_shuf = np.array(index_shuf)

X_train = []
Y_train = []
X_test = []
Y_test = []
j=0
for i in index_shuf:
    if (j < test_size):
        X_test.append(X_train_ordered[:,i])
        Y_test.append(Y_train_ordered[:,i])
    else:
        X_train.append(X_train_ordered[:,i])
        Y_train.append(Y_train_ordered[:,i])
    j = j + 1
X_test = np.array(X_test)
Y_test = np.array(Y_test)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(np.shape(X_train), np.shape(Y_train), np.shape(X_test), np.shape(Y_test))

mu = np.mean(X_train)
sigma = np.std(X_train)

X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma

print(np.std(X_train), np.std(X_test))

## Check Shuffle Maintained labelling

In [ ]:
import utils_intensity_map as uim

In [ ]:
ind_first = np.where(index_shuf == 0)

print(Y_train_ordered[:,0])
intensity_map_normalized2 = uim.xtrain2imap(X_train_ordered[:,0], LMAX, imap_nside)
hp.mollview(intensity_map_normalized2, unit="Deviation from Mean",flip="geo")

print(Y_train[ind_first[0]-test_size,:])
intensity_map_normalized2 = uim.xtrain2imap(X_train[ind_first[0]-test_size,:], LMAX, imap_nside)
hp.mollview(intensity_map_normalized2 * sigma + mu, unit="Deviation from Mean",flip="geo")

# Neural Network

In [ ]:
import tf_neural_network as tfnn
import matplotlib.pyplot as plt

In [ ]:
filename_nn_weights = "neural_network_weights"
input_size = num_coeff * 2
output_size = num_output
lr = 0.0001
num_epochs = 100
min_epoch = 10
costs = []
train_acc = []
test_acc = []
epoch_count = 0
hidden_units1 = 25
hidden_units2 = 20

In [ ]:
parameters, costs, train_acc, test_acc = tfnn.model_wrapper(X_train, Y_train, X_test, Y_test, input_size, output_size, learning_rate = lr, num_epochs = num_epochs, hidden_units1=hidden_units1, hidden_units2=hidden_units2)
nrw.save_nn_weights(parameters, filename_nn_weights)

epoch_count = num_epochs

In [ ]:
epochs = np.linspace(min_epoch, epoch_count, len(costs))

# Plot the cost
fig = plt.figure()
ax = plt.axes()

plt.plot(epochs, costs)
ax.set_xticks(range(0, epoch_count+int(epoch_count/10), int(epoch_count/10)))
plt.ylabel('cost')
plt.xlabel('epochs')
plt.title("Learning rate =" + str(lr))

plt.savefig(figure_location+"/cost_over_epochs.png", dpi=300, bbox_inches='tight')

## Open NN Weights

In [ ]:
parameters = nrw.read_nn_weights(filename_nn_weights)

## Restart Training

In [ ]:
num_epochs = 100

parameters, costs2, train_acc2, test_acc2 = tfnn.model_wrapper(X_train, Y_train, X_test, Y_test, input_size, output_size, learning_rate = lr, num_epochs = num_epochs, restart = True, nn_weights = parameters)
nrw.save_nn_weights(parameters, filename_nn_weights)

costs = np.append(costs, costs2)
train_acc = np.append(train_acc, train_acc2)
test_acc = np.append(test_acc, test_acc2)
epoch_count = epoch_count + num_epochs

In [ ]:
epochs = np.linspace(min_epoch, epoch_count, len(costs))

# Plot the cost
fig = plt.figure()
ax = plt.axes()

plt.plot(epochs, costs)
ax.set_xticks(range(0, epoch_count+int(epoch_count/10), int(epoch_count/10)))
plt.ylabel('cost')
plt.xlabel('epochs')
plt.title("Learning rate =" + str(lr))

plt.savefig(figure_location+"/cost_over_epochs.png", dpi=300, bbox_inches='tight')